In [6]:
import os
import pandas as pd
import numpy as np
import plotnine
from datetime import date
import scipy
import seaborn as sns
from Bio import SeqIO
import matplotlib.pyplot as plt
from upsetplot import from_memberships

In [7]:
## this notebook's purpose is to subset the assemblies by only contigs that are agreed upon
## by some number of assemblers.

In [17]:
curr_date="2022-01-24"
all_results = pd.read_csv(os.path.join("..","data-output", "all_results." + str(curr_date) + ".csv"))

In [8]:
## GLOBAL PARAMETERS ##
base_dir = "/vortexfs1/omics/alexander/akrinos/2021-testing-eukrhythmic/eukrhythmic_paper_trials_September21"
eukulele_dir_designer = "EUKulele_designer"
eukulele_dir_CAG = "EUKulele_07-CAG"
eukulele_dir_MAD = "EUKulele_12-MAD"
salmon_dir = os.path.join(base_dir,"08-salmon_mapping")

In [5]:
def count_set(x):
    x = x.dropna()
    y = []
    [y.extend(curr.split("-")) for curr in x]
    try:
        len(sorted(list(set(y))))
    except: 
        print(y,flush=True)
    return len(sorted(list(set(y))))
output_dir="two_way_contigs" #four_way_contigs
num_in_common = 2
os.system("mkdir -p " + output_dir)
## READ IN EUKRHYTHMIC DATA##
jeukebox_dirs = [curr for curr in os.listdir(base_dir) if "jEUKebox" in curr]
all_four_way_contigs = pd.DataFrame()
all_three_way_contigs = pd.DataFrame()
all_two_way_contigs = pd.DataFrame()
all_only_one_way = pd.DataFrame()

def create_set(x):
    x = x.dropna()
    y = []
    [y.extend(curr.split("-")) for curr in x]
    return "-".join(sorted(list(set(y))))

def count_set(x):
    x = x.dropna()
    y = []
    [y.extend(curr.split("-")) for curr in x]
    try:
        len(sorted(list(set(y))))
    except: 
        print(y,flush=True)
    return len(sorted(list(set(y))))

for jeukebox_dir in jeukebox_dirs:
    sourmash_file = os.path.join(base_dir, jeukebox_dir, "sourmash_MMETSP", "all.csv")
    if os.path.isfile(sourmash_file):
        
        if not os.path.isdir(os.path.join(base_dir, jeukebox_dir, 
                             "sourmash_MMETSP")):
            continue
        prot_clust_dir = os.path.join(base_dir, jeukebox_dir,
                                      "eukrhythmic_assembly",
                                      "intermediate-files",
                                      "03-merge", "07-CAG")
        if not os.path.isdir(prot_clust_dir):
            continue
            
        print(jeukebox_dir,"passed prot clust",flush=True)
        cluster_results = pd.DataFrame()
        for sim_file in os.listdir(prot_clust_dir):
            if "tsv" not in sim_file:
                continue
            fasta_file=sim_file.split(".tsv")[0] + ".fasta"
            ## WHAT'S INSIDE THE CAG FINAL CONTIGS??
            cluster_result = pd.read_csv(os.path.join(prot_clust_dir,
                                                      sim_file),sep="\t",
                                        header=None,names=["Representative","Member"])
            comm_num = int(sim_file.split("_merged")[0].split("_")[-1])
            cluster_result["Community"] = int(sim_file.split("_merged")[0].split("_")[-1])
            cluster_result["Name"] = jeukebox_dir
            cluster_result["Assembler_Rep"] = [curr.split("_")[0] for curr in cluster_result.Representative]
            cluster_result["Assembler_Mem"] = [curr.split("_")[0] for curr in cluster_result.Member]
            
            four_way_contigs = cluster_result.groupby(["Community","Name","Representative",
                                                       "Assembler_Rep"]).agg({"Assembler_Mem": [count_set,create_set]}).reset_index()
            four_way_contigs.columns = ['_'.join(col).strip() for col in four_way_contigs.columns.values]
            four_way_contigs_two = four_way_contigs[four_way_contigs["Assembler_Mem_count_set"] >= num_in_common]
            four_way_contigs_three = four_way_contigs[four_way_contigs["Assembler_Mem_count_set"] >= 3]
            four_way_contigs_four = four_way_contigs[four_way_contigs["Assembler_Mem_count_set"] >= 4]
            only_one_way_contigs = four_way_contigs[four_way_contigs["Assembler_Mem_count_set"] == 1]
            all_two_way_contigs= pd.concat([all_two_way_contigs,four_way_contigs_two])
            all_three_way_contigs= pd.concat([all_three_way_contigs,four_way_contigs_three])
            all_four_way_contigs= pd.concat([all_four_way_contigs,four_way_contigs_four])
            all_only_one_way = pd.concat([all_only_one_way,only_one_way_contigs])
            four_way_contigs_list = list(four_way_contigs["Representative_"])
            from Bio import SeqIO

            #with open(os.path.join(prot_clust_dir,fasta_file)) as input_handle, open(
            #    output_dir+"/" + jeukebox_dir + "_" + str(comm_num) + ".fasta", "w"
            #) as output_handle:
            #    sequences = [curr for curr in \
            #     SeqIO.parse(input_handle, "fasta") if curr.id in four_way_contigs_list]
            #    count = SeqIO.write(sequences, output_handle, "fasta")
            

jEUKebox-Trial3-CommB passed prot clust
jEUKebox-Trial2-CommB passed prot clust
jEUKebox-Trial4-100k passed prot clust
jEUKebox-Trial2-100k passed prot clust
jEUKebox-Trial1-CommB passed prot clust
jEUKebox-Trial3-100k passed prot clust
jEUKebox-Trial1-100k passed prot clust
jEUKebox-Trial4-CommB passed prot clust


In [9]:
all_seq_files = pd.DataFrame()

for jeukebox_dir in jeukebox_dirs:
    base_dir_covs = os.path.join(base_dir, jeukebox_dir, "clustering",
                                "clustering_sim_designer","prot-merge")
    coverage_dirs = os.listdir(base_dir_covs)
    for comm_num in range(1,7):
        for coverage in coverage_dirs:
            cov_curr = coverage.split("_")[-1]
            if float(cov_curr) != 0.9:
                continue
            seq_ids = os.listdir(os.path.join(base_dir_covs,coverage))
            for seq_id in seq_ids:
                seq_curr = seq_id.split("_")[-1]
                if float(seq_curr) != 0.9:
                    continue
                seq_file = pd.read_csv(os.path.join(os.path.join(base_dir_covs,coverage,
                                                                 seq_id,
                                                                 "sim_raw_reads_" + \
                                    str(comm_num) + \
                                    "_and_designer_assembly_" + \
                                    str(comm_num) + ".tsv")),sep="\t",header=None,names=["Representative","Member"])
                seq_file["Coverage"] = cov_curr
                seq_file["SeqID"] = seq_curr
                seq_file["jEUKebox"] = jeukebox_dir
                seq_file["Community"] = comm_num
                if "100k" in jeukebox_dir:
                    seq_file["MMETSPGroup"] = "A"
                else:
                    seq_file["MMETSPGroup"] = "B"
                seq_file["ShortMemName"] = [curr.split(".p")[0] + curr.split(".p")[1].split("_")[0] \
                                         if ".p" in curr else curr for curr in seq_file.Member]
                all_seq_files = pd.concat([all_seq_files,seq_file])

In [10]:
all_seq_files["ShortMemName"] = [curr.split(".p")[0] + ".p" + curr.split(".p")[1].split("_")[0] \
                                 if ".p" in curr else "_".join(curr.split("_")[0:2]) if ("CAMPEP" in\
                                 curr) & (len(curr.split("_")) > 1) else curr for curr in all_seq_files.Member]

In [11]:
all_seq_files["Assembler"] = [curr.split("_")[0] for curr in all_seq_files.Member]

## Grab the names of the CAG contigs that cluster w/ designer.

In [12]:
def create_set(x):
    x = x.dropna()
    y = []
    [y.extend(curr.split("-")) for curr in x]
    return "-".join(sorted(list(set(y))))

def count_set(x):
    x = x.dropna()
    y = []
    [y.extend(curr.split("-")) for curr in x]
    try:
        len(sorted(list(set(y))))
    except: 
        print(y,flush=True)
    return len(sorted(list(set(y))))

cluster_up = pd.merge(all_seq_files.groupby(["jEUKebox","Community","Representative",
                                    "MMETSPGroup"]).agg({"Assembler":[create_set]}).reset_index(),
                      all_seq_files, left_on = ["jEUKebox","Community","Representative",
                                    "MMETSPGroup"], 
                      right_on = ["jEUKebox","Community","Representative",
                                    "MMETSPGroup"])

/vortexfs1/home/akrinos/.local/lib/python3.8/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)


In [13]:
short_names = [".".join(curr_mem.split(".")[0:2]) if "spades" in curr_mem else curr_mem.split(".")[0] \
               for curr_mem, curr_set \
                          in zip(cluster_up.Member,cluster_up[("Assembler", 
                                                               "create_set")])]

clusters_with_des_df = pd.DataFrame({"ShortName": short_names,
                                     "Community": cluster_up.Community,
                                     "jEUKebox": cluster_up.jEUKebox,
                                    "inDesigner": ["inDes" if \
                          ("CAMPEP" in curr_set) & ("CAMPEP" not in curr_mem) else \
                                                   "not" for curr_mem, curr_set \
                          in zip(cluster_up.Member,cluster_up[("Assembler", 
                                                               "create_set")])]})

In [14]:
short_names = [".".join(curr_mem.split(".")[0:2]) if "spades" in curr_mem else curr_mem.split(".")[0] \
               for curr_mem, curr_set \
                          in zip(cluster_up.Member,cluster_up[("Assembler", 
                                                               "create_set")])]

clusters_with_des_df = pd.DataFrame({"ShortName": short_names,
                                     "Community": cluster_up.Community,
                                     "jEUKebox": cluster_up.jEUKebox,
                                    "inDesigner": ["inDes" if \
                          ("CAMPEP" in curr_set) & ("CAMPEP" not in curr_mem) else \
                                                   "not" for curr_mem, curr_set \
                          in zip(cluster_up.Member,cluster_up[("Assembler", 
                                                               "create_set")])]})

In [18]:
combined_clust_des = pd.merge(all_results, clusters_with_des_df, right_on=["ShortName",
                                                                           "Community",
                                                                           "jEUKebox"],
                              left_on=["Representative","Community","Name"],
                              how="left")

In [19]:
combined_clust_des.loc[combined_clust_des.inDesigner != combined_clust_des.inDesigner,"inDesigner"] = "noProtein"

In [20]:
def create_set(x):
    x = x.dropna()
    y = []
    [y.extend(curr.split("-")) for curr in x]
    return "-".join(sorted(list(set(y))))

def count_set(x):
    x = x.dropna()
    y = []
    [y.extend(curr.split("-")) for curr in x]
    try:
        len(sorted(list(set(y))))
    except: 
        print(y,flush=True)
    return len(sorted(list(set(y))))

count_assembler_clust = combined_clust_des.groupby(["Name","Community",
                                                    "Representative","inDesigner"]).\
    agg({"Assembler_Mem":[create_set,count_set]})

In [21]:
count_assembler_clust.to_csv(os.path.join("..","data-output", 
                                    "count_assembler_clust." + str(date.today()) + ".csv"))

In [22]:
count_assembler_clust_reset = count_assembler_clust.reset_index()

count_assembler_clust_reset.columns = [' '.join(col).strip() for col in count_assembler_clust_reset.columns.values]

counted_conds = count_assembler_clust_reset.groupby(["Name","Community",
                                                     "inDesigner","Assembler_Mem create_set",
                                     "Assembler_Mem count_set"])["Representative"].count()



In [23]:
counted_conds = counted_conds.reset_index()
counted_conds["AssemblerMemCtCat"] = counted_conds["Assembler_Mem count_set"].astype(str)

In [24]:
compare_conds_tab = counted_conds.pivot_table(values = "Representative",
                          index = ["Name","Community","Assembler_Mem create_set",
                                    "AssemblerMemCtCat","Assembler_Mem count_set"],
                          columns = "inDesigner").reset_index()
compare_conds_tab["PropSpurious"] = compare_conds_tab["not"] / \
    (compare_conds_tab["inDes"] + compare_conds_tab["not"])

In [25]:
compare_conds_tab.to_csv(os.path.join("..","data-output", 
                                      "compare_conds_tab." + str(date.today()) + ".csv"))

In [26]:
all_only_one_way.head()

,Community_,Name_,Representative_,Assembler_Rep_,Assembler_Mem_count_set,Assembler_Mem_create_set
0,2,jEUKebox-Trial3-CommB,megahit_sim_raw_reads_2_k51_10002,megahit,1,megahit
1,2,jEUKebox-Trial3-CommB,megahit_sim_raw_reads_2_k51_10012,megahit,1,megahit
2,2,jEUKebox-Trial3-CommB,megahit_sim_raw_reads_2_k51_1002,megahit,1,megahit
4,2,jEUKebox-Trial3-CommB,megahit_sim_raw_reads_2_k51_10042,megahit,1,megahit
5,2,jEUKebox-Trial3-CommB,megahit_sim_raw_reads_2_k51_10045,megahit,1,megahit


In [27]:
all_seq_files.head()

,Representative,Member,Coverage,SeqID,jEUKebox,Community,MMETSPGroup,ShortMemName,Assembler
0,CAMPEP_0203267998_/NCGR_PEP_ID=MMETSP1411-2013...,CAMPEP_0203267998_/NCGR_PEP_ID=MMETSP1411-2013...,0.9,0.9,jEUKebox-Trial3-CommB,1,B,CAMPEP_0203267998,CAMPEP
1,trinity_sim_raw_reads_1_TRINITY_DN14073_c0_g1_...,trinity_sim_raw_reads_1_TRINITY_DN14073_c0_g1_...,0.9,0.9,jEUKebox-Trial3-CommB,1,B,trinity_sim_raw_reads_1_TRINITY_DN14073_c0_g1_...,trinity
2,trinity_sim_raw_reads_1_TRINITY_DN15616_c0_g1_...,trinity_sim_raw_reads_1_TRINITY_DN15616_c0_g1_...,0.9,0.9,jEUKebox-Trial3-CommB,1,B,trinity_sim_raw_reads_1_TRINITY_DN15616_c0_g1_...,trinity
3,trinity_sim_raw_reads_1_TRINITY_DN15722_c0_g1_...,trinity_sim_raw_reads_1_TRINITY_DN15722_c0_g1_...,0.9,0.9,jEUKebox-Trial3-CommB,1,B,trinity_sim_raw_reads_1_TRINITY_DN15722_c0_g1_...,trinity
4,trinity_sim_raw_reads_1_TRINITY_DN15839_c0_g1_...,trinity_sim_raw_reads_1_TRINITY_DN15839_c0_g1_...,0.9,0.9,jEUKebox-Trial3-CommB,1,B,trinity_sim_raw_reads_1_TRINITY_DN15839_c0_g1_...,trinity
